<table align="left">
  <tr>
 <td><img src="icecreamtruck.jpeg" alt="Ice Cream Truck" width="150"/></td>
 <td align="left"><h1>Lesson 2: Introduction to Code Agents</h1></td>

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code>  file:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

In [1]:
# Warning control
import warnings

warnings.filterwarnings("ignore")

import os
import io
import IPython.display
from PIL import Image
import base64

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv() # read local .env file

from huggingface_hub import login

login(os.environ['HF_API_KEY'])

In [2]:
import pandas as pd

suppliers_data = {
    "name": [
        "Montreal Ice Cream Co",
        "Brain Freeze Brothers",
        "Toronto Gelato Ltd",
        "Buffalo Scoops",
        "Vermont Creamery",
    ],
    "location": [
        "Montreal, QC",
        "Burlington, VT",
        "Toronto, ON",
        "Buffalo, NY",
        "Portland, ME",
    ],
    "distance_km": [120, 85, 400, 220, 280],
    "canadian": [True, False, True, False, False],
    "price_per_liter": [1.95, 1.91, 1.82, 2.43, 2.33],
    "tasting_fee": [0, 12.50, 30.14, 42.00, 0.20],
}

data_description = """Suppliers have an additional tasting fee: that is a fixed fee applied to each order to taste the ice cream."""
suppliers_df = pd.DataFrame(suppliers_data)
suppliers_df

,name,location,distance_km,canadian,price_per_liter,tasting_fee
0,Montreal Ice Cream Co,"Montreal, QC",120,True,1.95,0.00
1,Brain Freeze Brothers,"Burlington, VT",85,False,1.91,12.50
2,Toronto Gelato Ltd,"Toronto, ON",400,True,1.82,30.14
3,Buffalo Scoops,"Buffalo, NY",220,False,2.43,42.00
4,Vermont Creamery,"Portland, ME",280,False,2.33,0.20


## Setup tools

In [3]:
import numpy as np

def calculate_daily_supplier_price(row):
    order_volume = 30

    # Calculate raw product cost
    product_cost = row["price_per_liter"] * order_volume

    # Calculate transport cost
    trucks_needed = np.ceil(order_volume / 300)
    cost_per_km = 1.20
    transport_cost = row["distance_km"] * cost_per_km * trucks_needed

    # Calculate tariffs for ice cream imported from Canada
    tariff = product_cost * np.pi / 50 * row["canadian"]

    # Get total cost
    total_cost = product_cost + transport_cost + tariff + row["tasting_fee"]
    return total_cost

suppliers_df["daily_price"] = suppliers_df.apply(calculate_daily_supplier_price, axis=1)
display(suppliers_df)

# Let's remove this column now.
suppliers_df = suppliers_df.drop("daily_price", axis=1)

,name,location,distance_km,canadian,price_per_liter,tasting_fee,daily_price
0,Montreal Ice Cream Co,"Montreal, QC",120,True,1.95,0.00,206.175663
1,Brain Freeze Brothers,"Burlington, VT",85,False,1.91,12.50,171.800000
2,Toronto Gelato Ltd,"Toronto, ON",400,True,1.82,30.14,568.170619
3,Buffalo Scoops,"Buffalo, NY",220,False,2.43,42.00,378.900000
4,Vermont Creamery,"Portland, ME",280,False,2.33,0.20,406.100000


In [4]:
# First we make a few tools
from smolagents import tool

@tool
def calculate_transport_cost(distance_km: float, order_volume: float) -> float:
    """
    Calculate transportation cost based on distance and order size.
    Refrigerated transport costs $1.2 per kilometer and has a capacity of 300 liters.

    Args:
        distance_km: the distance in kilometers
        order_volume: the order volume in liters
    """
    trucks_needed = np.ceil(order_volume / 300)
    cost_per_km = 1.20
    return distance_km * cost_per_km * trucks_needed


@tool
def calculate_tariff(base_cost: float, is_canadian: bool) -> float:
    """
    Calculates tariff for Canadian imports. Returns the tariff only, not the total cost.
    Assumes tariff on dairy products from Canada is worth 2 * pi / 100, approx 6.2%

    Args:
        base_cost: the base cost of goods, not including transportation cost.
        is_canadian: wether the import is from Canada.
    """
    if is_canadian:
        return base_cost * np.pi / 50
    return 0

In [5]:
calculate_transport_cost.description

'Calculate transportation cost based on distance and order size.\nRefrigerated transport costs $1.2 per kilometer and has a capacity of 300 liters.'

## Setup the Model

In [6]:
from smolagents import HfApiModel, CodeAgent
from helper import get_huggingface_token

model = HfApiModel(
    "Qwen/Qwen2.5-72B-Instruct",
    provider="together", # Choose a specific inference provider
    max_tokens=4096,
    temperature=0.1
)

## Setup the Code Agent

In [7]:
agent = CodeAgent(
    model=model,
    tools=[calculate_transport_cost, calculate_tariff],
    max_steps=10,
    additional_authorized_imports=["pandas", "numpy"],
    verbosity_level=2
)
agent.logger.console.width=66

In [8]:
agent.run(
    """Can you get me the transportation cost for 50 liters
    of ice cream over 10 kilometers?"""
)

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ Can you get me the transportation cost for 50 liters           │
│     of ice cream over 10 kilometers?                           │
│                                                                │
╰─ HfApiModel - Qwen/Qwen2.5-72B-Instruct ───────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ───────────────────────────────────────
Thought: I will use the `calculate_transport_cost` tool to        
calculate the transportation cost for 50 liters of ice cream over 
10 kilometers.                                                    
Code:                                                             
```py                                                             
transport_cost = calculate_transport_cost(distance_km=10,         
order_volume=50)                                                  
print(transport_cost)                                             
```<end_code>                                                     

─ Executing parsed code: ─────────────────────────────────────── 
  transport_cost = calculate_transport_cost(distance_km=10,       
  order_volume=50)                                                
  print(transport_cost)                                           
 ────────────────────────────────────────────────────────────────

Execution logs:
12.0

Out: None

...

Output message of the LLM: ───────────────────────────────────────
Thought: The transportation cost for 50 liters of ice cream over  
10 kilometers is 12.0. I will provide this as the final answer.   
Code:                                                             
```py                                                             
final_answer(12.0)                                                
```<end_code>                                                     

─ Executing parsed code: ─────────────────────────────────────── 
  final_answer(12.0)                                              
 ────────────────────────────────────────────────────────────────

Out - Final answer: 12.0

[Step 2: Duration 0.02 seconds| Input tokens: 4,638 | Output 
tokens: 116]

12.0

## Give the agent detailed instructions

In [9]:
task = """Here is a dataframe of different ice cream suppliers.
Could you give me a comparative table (as a dataframe) of the total
daily price for getting daily ice cream delivery from each of them,
given that we need exactly 30 liters of ice cream per day? Take
into account transportation cost and tariffs.
"""
agent.logger.level = 1 # Lower verbosity level
agent.run(
    task,
    additional_args={"suppliers_data": suppliers_df, "data_description": data_description},
)

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ Here is a dataframe of different ice cream suppliers.          │
│ Could you give me a comparative table (as a dataframe) of the  │
│ total                                                          │
│ daily price for getting daily ice cream delivery from each of  │
│ them,                                                          │
│ given that we need exactly 30 liters of ice cream per day?     │
│ Take                                                           │
│ into account transportation cost and tariffs.                  │
│                                                                │
│ You have been provided with these additional arguments, that   │
│ you can access using the keys as variables in your python      │
│ code:                                                          │
│ {'suppliers_data':                     name        location    │
│ distance_km  canadian  \                                       │
│ 0  Montreal Ice Cream Co    Montreal, QC          120          │
│ True                                                           │
│ 1  Brain Freeze Brothers  Burlington, VT           85          │
│ False                                                          │
│ 2     Toronto Gelato Ltd     Toronto, ON          400          │
│ True                                                           │
│ 3         Buffalo Scoops     Buffalo, NY          220          │
│ False                                                          │
│ 4       Vermont Creamery    Portland, ME          280          │
│ False                                                          │
│                                                                │
│    price_per_liter  tasting_fee                                │
│ 0             1.95         0.00                                │
│ 1             1.91        12.50                                │
│ 2             1.82        30.14                                │
│ 3             2.43        42.00                                │
│ 4             2.33         0.20  , 'data_description':         │
│ 'Suppliers have an additional tasting fee: that is a fixed fee │
│ applied to each order to taste the ice cream.'}.               │
│                                                                │
╰─ HfApiModel - Qwen/Qwen2.5-72B-Instruct ───────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ─────────────────────────────────────── 
  import pandas as pd                                             
                                                                  
  # Convert the provided data into a DataFrame                    
  suppliers_df = pd.DataFrame(suppliers_data)                     
                                                                  
  # Calculate the transportation cost for each supplier           
  suppliers_df['transport_cost'] = suppliers_df.apply(            
      lambda row:                                                 
  calculate_transport_cost(distance_km=row['distance_km'],        
  order_volume=30), axis=1                                        
  )                                                               
                                                                  
  # Calculate the tariff for each supplier                        
  suppliers_df['tariff'] = suppliers_df.apply(                    
      lambda row:                                                 
  calculate_tariff(base_cost=row['price_per_liter'] * 30,         
  is_canadian=row['canadian']), axis=1                            
  )                                                               
                                                                  
  # Calculate the total daily price for each supplier             
  suppliers_df['total_daily_price'] = (                           
      suppliers_df['price_per_liter'] * 30 +  # Cost of 30        
  liters of ice cream                                             
      suppliers_df['tasting_fee'] +  # Tasting fee                
      suppliers_df['transport_cost'] +  # Transportation cost     
      suppliers_df['tariff']  # Tariff                            
  )                                                               
                                                                  
  # Select the relevant columns for the final output              
  comparative_table = suppliers_df[['name',                       
  'total_daily_price']]                                           
                                                                  
  print(comparative_table)                                        
 ────────────────────────────────────────────────────────────────

Execution logs:
                    name  total_daily_price
0  Montreal Ice Cream Co         206.175663
1  Brain Freeze Brothers         171.800000
2     Toronto Gelato Ltd         568.170619
3         Buffalo Scoops         378.900000
4       Vermont Creamery         406.100000

Out: None

[Step 1: Duration 0.16 seconds| Input tokens: 2,517 | Output 
tokens: 359]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ─────────────────────────────────────── 
  final_answer(comparative_table)                                 
 ────────────────────────────────────────────────────────────────

Out - Final answer:                     name  total_daily_price
0  Montreal Ice Cream Co         206.175663
1  Brain Freeze Brothers         171.800000
2     Toronto Gelato Ltd         568.170619
3         Buffalo Scoops         378.900000
4       Vermont Creamery         406.100000

[Step 2: Duration 0.02 seconds| Input tokens: 5,815 | Output 
tokens: 404]

,name,total_daily_price
0,Montreal Ice Cream Co,206.175663
1,Brain Freeze Brothers,171.800000
2,Toronto Gelato Ltd,568.170619
3,Buffalo Scoops,378.900000
4,Vermont Creamery,406.100000


## Compare to traditional tool calling.

In [10]:
from smolagents import ToolCallingAgent

model = HfApiModel(
    "Qwen/Qwen2.5-72B-Instruct",
    temperature=0.6
)
agent = ToolCallingAgent(
    model=model,
    tools=[calculate_transport_cost, calculate_tariff],
    max_steps=20,
)
agent.logger.console.width=66

<div style="font-size: 14px; background-color: #fff9b0; padding: 12px; border-left: 4px solid #facc15;">
  <strong>Note:</strong> During this run, the agent may generate code that creates an error. This is OK. The agent will try to fix the error.
</div>


In [11]:
output = agent.run(
    task,
    additional_args={"suppliers_data": suppliers_df, "data_description": data_description},
)
print(output)

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ Here is a dataframe of different ice cream suppliers.          │
│ Could you give me a comparative table (as a dataframe) of the  │
│ total                                                          │
│ daily price for getting daily ice cream delivery from each of  │
│ them,                                                          │
│ given that we need exactly 30 liters of ice cream per day?     │
│ Take                                                           │
│ into account transportation cost and tariffs.                  │
│                                                                │
│ You have been provided with these additional arguments, that   │
│ you can access using the keys as variables in your python      │
│ code:                                                          │
│ {'suppliers_data':                     name        location    │
│ distance_km  canadian  \                                       │
│ 0  Montreal Ice Cream Co    Montreal, QC          120          │
│ True                                                           │
│ 1  Brain Freeze Brothers  Burlington, VT           85          │
│ False                                                          │
│ 2     Toronto Gelato Ltd     Toronto, ON          400          │
│ True                                                           │
│ 3         Buffalo Scoops     Buffalo, NY          220          │
│ False                                                          │
│ 4       Vermont Creamery    Portland, ME          280          │
│ False                                                          │
│                                                                │
│    price_per_liter  tasting_fee                                │
│ 0             1.95         0.00                                │
│ 1             1.91        12.50                                │
│ 2             1.82        30.14                                │
│ 3             2.43        42.00                                │
│ 4             2.33         0.20  , 'data_description':         │
│ 'Suppliers have an additional tasting fee: that is a fixed fee │
│ applied to each order to taste the ice cream.'}.               │
│                                                                │
╰─ HfApiModel - Qwen/Qwen2.5-72B-Instruct ───────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error while generating or parsing output:
404 Client Error: Not Found for url: 
http://jupyter-api-proxy.internal.dlai/rev-proxy/huggingface_infer
ence_provider/hf-inference/models/Qwen/Qwen2.5-72B-Instruct/v1/cha
t/completions (Request ID: 
Self=1-688569e6-5958f3b6114e725b224cb1ef;Root=1-688569e6-4b59daeb7
2477e6f50ee8080;4da6f5e4-de1b-4936-ac11-ac8f7a55ff5b)

[Step 1: Duration 0.28 seconds]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

Error while generating or parsing output:
404 Client Error: Not Found for url: 
http://jupyter-api-proxy.internal.dlai/rev-proxy/huggingface_infer
ence_provider/hf-inference/models/Qwen/Qwen2.5-72B-Instruct/v1/cha
t/completions (Request ID: 
Self=1-688569e9-71a4327f297336752d5429df;Root=1-688569e9-686389814
22be51852ac87cd;2acca5c0-d005-4142-b6c2-cd88b4942d08)

[Step 20: Duration 0.15 seconds]

Reached max steps.

[Step 21: Duration 0.49 seconds]

Error in generating final LLM output:
404 Client Error: Not Found for url: http://jupyter-api-proxy.internal.dlai/rev-proxy/huggingface_inference_provider/hf-inference/models/Qwen/Qwen2.5-72B-Instruct/v1/chat/completions (Request ID: Self=1-688569e9-5951c4ea033aeb273d9ee923;Root=1-688569e9-0832345f5e413dc75ebca848;7075dafe-ff3a-4ecb-98c2-f9899f788bea)
